In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())) + "/..")
if os.getcwd().endswith("Topic2"):
    os.chdir("..")
    print(f"Changed to {os.getcwd()}")

from codebase import load_excel

Changed to c:\Users\toell\Documents\GitHub\Design-of-AI-driven-business-operation


In [4]:
odin_excel_path = os.path.join(os.getcwd(), "data", "OdiN 2019-2023", "OdiN 2023", "ODiN2023_Databestand.csv")
df = load_excel(odin_excel_path)

Loaded file from CSV


In [ ]:
def make_ml_dataset(df, target_col, drop_cols, target_val=None, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets.
    """
    from sklearn.model_selection import train_test_split

    # Drop specified columns
    df = df.drop(columns=drop_cols)

    # Split the data into features and target
    X = df.drop(columns=[target_col])
    y = df[target_col] == target_val if target_val is not None else df[target_col]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

In [7]:
print(list(df.columns))

['OP', 'OPID', 'Steekproef', 'Mode', 'HHPers', 'HHSam', 'HHPlOP', 'HHLft1', 'HHLft2', 'HHLft3', 'HHLft4', 'WoPC', 'WoGem', 'Sted', 'GemGr', 'Prov', 'Corop', 'BuurtAdam', 'MRA', 'MRDH', 'Utr', 'Geslacht', 'Leeftijd', 'KLeeft', 'HerkLand', 'BetWerk', 'OnbBez', 'MaatsPart', 'Opleiding', 'HHBestInkG', 'HHGestInkG', 'HHLaagInk', 'HHSocInk', 'HHWelvG', 'HHRijbewijsAu', 'HHRijbewijsMo', 'HHRijbewijsBr', 'OPRijbewijsAu', 'OPRijbewijsMo', 'OPRijbewijsBr', 'HHAuto', 'HHAutoL', 'OPAuto', 'BrandstofPa1', 'XBrandstofPa1', 'BrandstofEPa1', 'BouwjaarPa1', 'KBouwjaarPa1', 'KGewichtPa1', 'TenaamPa1', 'BrandstofPa2', 'XBrandstofPa2', 'BrandstofEPa2', 'BouwjaarPa2', 'KBouwjaarPa2', 'KGewichtPa2', 'TenaamPa2', 'BrandstofPaL', 'XBrandstofPaL', 'BrandstofEPaL', 'BouwjaarPaL', 'KBouwjaarPaL', 'KGewichtPaL', 'HHMotor', 'OPMotor', 'HHBrom', 'OPBrom', 'HHSnor', 'OPSnor', 'HHEFiets', 'HHBezitVm', 'OPBezitVm', 'FqLopen', 'FqNEFiets', 'FqEFiets', 'FqBTM', 'FqTrein', 'FqAutoB', 'FqAutoP', 'FqBrSnor', 'FqMotor', 'OV

In [ ]:
drop_cals = [
    "OP", 
    "OPID",
    "Steekproef", 
    "Mode",
    "Corop",
    "BuurtAdam",
    "KLeeft",
    "Jaar",
    "Maand",
    "Week",
    "Dag",
    "Weekdag",
    "Feestdag",
    # Not sure bc of moving??:
    "Verpl",
    "VerplID",
    "VerplNr",
    "Toer",
    "AantRit"
    # End of not sure
    ]

numerical_cols = [
    "HHPers",
    "HHLft1",
    "HHLft2",
    "HHLft3",
    "HHLft4",
    "Leeftijd",
    "HHRijbweijsAu",
    "HHRijbewijsMo",
    "HHRijbewijsBr",
    "HHAuto",
    "HHAutoL",
    "OPAuto",
    "BouwjaarPa1",
    "BouwjaarPa2",
    "BouwjaarPaL",
    "HHMotor",
    "OPBrom",
    "HHSnor",
    "OPSnor",
    "HHFiets",
    "ReisduurOP",
    "AfstandOP",
    "AfstandSOP",
]

ordinal_cols = [
    "HHBestInkG",
    "HHGestInkG",
    "HHLaagInk",
    "HHSocInk",
    "HHWelvG",
    "KBouwjaarPa1",
    "KBouwjaarPa2",
    "KGewitchPa1",
    "KGewichtPa2",
    "KBouwjaarPaL",
    "KGewichtPaL",
    "FqLopen",
    "FqNEFiets",
    "FqEFiets",
    "FqBTM",
    "FqTrein",
    "FqAutoB",
    "FqAutoP",
    "FqMotor",
    "FqBrSnor",
    "BerWrk",
    "BerOnd",
    "AantVpl",
    "AantOVVPl",
    "AantSVpl",

]

categorical_cols = [
    "HHSam",
    "HHPlOP",
    "WoPC",
    "Wogem",
    "Sted",
    "GemGr",
    "Prov",
    "MRA",
    "MRDH",
    "Utr",
    "Geslacht",
    "Herkomst",
    "BetWerk",
    "OnbBez",
    "MaatsPart",
    "Opleiding",
    "BrandstofPa1",
    "XBrandstofPa1",
    "BrandstofEPa1",
    "BrandstofPaL",
    "XBrandstofPaL",
    "BrandstofEPaL",
    "TenaamPa2",
    "HHBezitsVim",
    "OPBezitsVim",
    "OVStKaart",
    # Yes/no/not relevant:
    "WrkVerg",
    "VergVast",
    "VergKm",
    "VergBrSt",
    "VergOV",
    "VergAans",
    "VergVoer",
    "VergBudg",
    "VergPark",
    "VergStal",
    "VergAnd",
    # End of yes/no/not relevant
    "RdWrkA",
    "RdWrkB",
    "RdOndA",
    "RdOndB",
    "BerSup",
    "RdSupA",
    "RdSupB",
    "BerZiek",  # last 3 are unknown, rest ranked (Ber seems to be the prefix for this)
    "RdZiekA",
    "RdZiekB",
    "BerArts",  # last 3 are unknown, rest ranked
    "RdArtsA",
    "RdArtsB",
    "BerStat",  # last 3 are unknown, rest ranked
    "RdStatA",
    "RdStatB",
    "BerHalte",  # last 3 are unknown, rest ranked
    "RDHalteA",
    "BerFam",
    "RdFamA",
    "RdFamB",
    "BerSport",
    "RdSportA",
    "RdSportB",
    "Weggeweest",
    "RedenNW",
    "RedenNWZ",
    "RedenNWW",
    "RedenNWB",
    "EFiets",
    # 0, 3, 4 unknownm 1, 2 are yes/no
    "AutoEig",
    "AutoHhl",
    "AutoLWg",
    "AutoLPl",
    "AutoBed",
    "AutoDOrg",
    "AutoDPart",
    "AutoDBek",
    "AutoLeen",
    "AutoHuur",
    "AutoAnd",
    "ByzDag",
    "ByzAdr",
    "ByzVvm",
    "ByzTyd",
    "ByzDuur",
    "ByzRoute",
    "ByzReden",
    "Doel",
    "MotiefV",
    "KMotiefV",
    "MeerWink",     # 0, 1 yes/no, 2, 3 unknown
    "AardWerk",
]

binary_cols = [
    "OPRijbewijsAu",
    "OPRijbewijsMo",
    "OPRijbewijsBr"
]

target_col = "WrkVervw"
target_val = 2

X_train, X_test, y_train, y_test = make_ml_dataset(df, target_col="target", drop_cols=drop_cals)

In [ ]:
# Use a random forest classifier as an example
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix



,OP,OPID,Steekproef,Mode,HHPers,HHSam,HHPlOP,HHLft1,HHLft2,HHLft3,...,RReisduur,RReisduurBL,RVertStat,RAankStat,RTSamen,RCorrSnelh,RVliegVer,FactorH,FactorP,FactorV
0,1,270000762969,4,1,3,3,3,0,0,0,...,20.0,0.0,0.0,0.0,13.0,0.0,0.0,79.014958,237.044874,86521.379089
1,0,270000762969,4,1,3,3,3,0,0,0,...,15.0,0.0,0.0,0.0,13.0,0.0,0.0,79.014958,237.044874,86521.379089
2,0,270000762969,4,1,3,3,3,0,0,0,...,2.0,0.0,0.0,0.0,13.0,0.0,0.0,79.014958,237.044874,86521.379089
3,0,270000762969,4,1,3,3,3,0,0,0,...,2.0,0.0,0.0,0.0,13.0,0.0,0.0,79.014958,237.044874,86521.379089
4,0,270000762969,4,1,3,3,3,0,0,0,...,30.0,0.0,0.0,0.0,13.0,0.0,0.0,79.014958,237.044874,86521.379089
